# Initial load of program

### Imports

In [1]:
import numpy as np
import pandas as pd
import regex as re
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper
import warnings

warnings.filterwarnings('ignore')

### Build graph around item

In [46]:
def build_graph(node: str):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(f"""
        CONSTRUCT {{ {node} ?p ?object .
                    ?object ?p1 ?subject1 .
                    ?subject ?p2 {node} .
                    ?subject2 ?p3 ?subject .
                    ?subject3 ?p4 ?object . 
                    ?subject ?p5 ?subject4 }} 
        WHERE       {{
                    {{ {node} ?p ?object .
                    ?object ?p1 ?subject1 .
                    ?subject3 ?p4 ?object .
                    FILTER isURI(?object)
                    FILTER isURI(?subject1)
                    FILTER isURI(?subject3) }}
                    UNION
                    {{ ?subject ?p2 {node} .
                    ?subject2 ?p3 ?subject .
                    ?subject ?p5 ?subject4 . 
                    FILTER isURI(?subject)
                    FILTER isURI(?subject2)
                    FILTER isURI(?subject4) }}
                    }}
                """)
    query_result = sparql.queryAndConvert()  # Produces an RDFlib Graph object 
    query_df = pd.DataFrame(query_result, columns=["Subject", "Property", "Value"])
    query_df["Subject"] = query_df["Subject"].str.replace("rdflib.term.URIRef", "")
    query_df["Value"] = query_df["Value"].str.replace("rdflib.term.URIRef", "")
    discard_properties = ["P921", "P2860"]
    query_df = query_df[~query_df.Property.str.contains('|'.join(discard_properties))]

    G = nx.Graph()
    G = nx.from_pandas_edgelist(query_df, "Subject", "Value")
    return G
    
test = build_graph("wd:Q171076")

### Finding other nodes with common neighbors 

In [53]:
def common_neighbors(graph: object, source_node: str):
    neighbor_dict = {}  # Dict to store nodes and the number of common neighbors with the source-node
    source_node_neighbors = [n for n in graph.neighbors(source_node)]  # List containing the source-node's neighbors 
    
    for item in source_node_neighbors:
        item_neighbors = [n for n in graph.neighbors(item)]
        common_neighbors = sum(n in source_node_neighbors for n in item_neighbors)
        neighbor_dict[item] = common_neighbors
    return sorted(neighbor_dict.items(), key=lambda item: item[1], reverse=True)

common_neighbors(test, "http://www.wikidata.org/entity/Q171076")

[('http://www.wikidata.org/entity/Q555853', 3),
 ('http://www.wikidata.org/entity/Q65464996', 3),
 ('http://www.wikidata.org/entity/Q65464991', 2),
 ('http://www.wikidata.org/entity/Q12705', 2),
 ('http://www.wikidata.org/entity/Q106595715', 2),
 ('http://www.wikidata.org/entity/Q1191614', 2),
 ('http://www.wikidata.org/entity/Q14515877', 2),
 ('http://www.wikidata.org/entity/Q9177986', 2),
 ('http://www.wikidata.org/entity/statement/Q171076-659DE9FF-C351-456C-B2E4-7FED7C2293CE',
  1),
 ('http://www.wikidata.org/entity/statement/Q2488244-f732663b-443f-0f07-6141-111d4d72fa51',
  1),
 ('http://www.wikidata.org/entity/statement/Q171076-a294ac74-4d51-c2ad-d81d-3d69adcbd529',
  1),
 ('http://www.wikidata.org/entity/statement/L30165-S1-58E03A0E-688E-4DA8-B1D5-2A7B30FD340A',
  1),
 ('http://www.wikidata.org/entity/statement/Q1191614-fd814a87-4757-1f54-8c1b-a8c1e1e6a1ec',
  1),
 ('http://www.wikidata.org/entity/statement/Q9177986-9A28ED15-0E69-46FD-95E2-ED3A5659163F',
  1),
 ('http://www.wikid

### Alternative

In [63]:
def common_neighbors_2(graph: object, node: str):
    node_dict = {}
    
    for item in graph.neighbors(node):
        for neighbor in graph.neighbors(item):
            if neighbor == node:
                continue
            common_neighbors = sum(n in graph.neighbors(neighbor) for n in graph.neighbors(node))
            node_dict[neighbor] = common_neighbors
    return sorted(node_dict.items(), key=lambda item: item[1], reverse=True)

common_neighbors_2(test, "http://www.wikidata.org/entity/Q171076")            

[('http://wikiba.se/ontology#BestRank', 52),
 ('http://wikiba.se/ontology#NormalRank', 52),
 ('http://www.w3.org/ns/lemon/ontolex#LexicalSense', 4),
 ('http://www.wikidata.org/entity/Q555853', 3),
 ('http://www.wikidata.org/entity/Q65464996', 3),
 ('http://www.wikidata.org/reference/c13ba4d7902ead2693f79396bc25bd35d4d14030',
  2),
 ('http://www.wikidata.org/reference/3bf39867b037e8e494a8389ae8a03bad6825a7fc',
  2),
 ('http://www.wikidata.org/entity/Q9177986', 2),
 ('http://www.wikidata.org/entity/Q900279', 2),
 ('http://www.wikidata.org/entity/Q1191614', 2),
 ('http://www.wikidata.org/entity/Q35', 2),
 ('http://www.wikidata.org/entity/statement/Q65464991-30d982a0-4949-4055-97cd-b98bd5f05738',
  2),
 ('http://www.wikidata.org/entity/statement/Q14515877-76f1a345-429b-b3b1-3670-4b061d90cd54',
  2),
 ('http://www.wikidata.org/entity/Q14515877', 2),
 ('http://www.wikidata.org/entity/Q4830453', 2),
 ('http://www.wikidata.org/entity/Q183', 2),
 ('http://www.wikidata.org/entity/Q106595715', 2)